In [1]:
stock = 'FB'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    209.940002
2018-07-23    210.910004
2018-07-24    214.669998
2018-07-25    217.500000
2018-07-26    176.259995
2018-07-27    174.889999
2018-07-30    171.059998
2018-07-31    172.580002
2018-08-01    171.649994
2018-08-02    176.369995
2018-08-03    177.779999
2018-08-06    185.690002
2018-08-07    183.809998
2018-08-08    185.179993
2018-08-09    183.089996
2018-08-10    180.259995
2018-08-13    180.050003
2018-08-14    181.110001
2018-08-15    179.529999
2018-08-16    174.699997
2018-08-17    173.800003
2018-08-20    172.500000
2018-08-21    172.619995
2018-08-22    173.639999
2018-08-23    172.899994
2018-08-24    174.649994
2018-08-27    177.460007
2018-08-28    176.259995
2018-08-29    175.899994
2018-08-30    177.639999
                 ...    
2018-09-17    160.580002
2018-09-18    160.300003
2018-09-19    163.059998
2018-09-20    166.020004
2018-09-21    162.929993
2018-09-24    165.410004
2018-09-25    164.910004
2018-09-26    166.949997
2018-09-27    168.83

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


149.57032766729114
[150.39443142 151.1908898  151.24515573 146.18617002 148.83499136]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")